# Image Offsets: Process sequence

This notebook transforms the procedures developed in `Offsets_1.ipynb` into callable functions. 

These functions are used in a loop to process an entire sequence of images. Results are later examined in plots. 

The end product is a series of FITS tables, one per input image, that contain the star offsets in relation to the reference image. These tables should be used in a subsequent notebook to generate the actual arrays with pixel offsets that are used by drizzle to figure out the pixel mapping.

In [ ]:
%pylab notebook
%matplotlib notebook

import os, glob

import numpy as np
from matplotlib.pyplot import imshow
import matplotlib.pyplot as plt

from scipy.optimize import minimize
from astropy.table import Table, vstack
from astropy.stats import SigmaClip

import photutils
from photutils import Background2D, ModeEstimatorBackground, DAOStarFinder, CircularAperture

import rawpy
import exifread

## Initialization

Define values to be used in the processing functions, and throughout the script.

In [ ]:
dirpath = '../astrophotography_data/MilkyWayPrettyBoy/12800/light/'
file_list = glob.glob(dirpath + '/*.ARW')

In [ ]:
# parameters for background subtraction and star finding
bkg_sigma = 3.0
bkg_cell_footprint = (50, 50)
bkg_filter = (5, 5)
dao_fwhm = 4.3
dao_threshold = 5.0
proximity = 2.5

# operators
sigma_clip = SigmaClip(sigma=bkg_sigma)
bkg_estimator = ModeEstimatorBackground()

In [ ]:
# 1st test image - this will be the reference image which subsequent images
# will have their offsets computed against. We need to read it here to get 
# the camera color array specification as well.
fname = file_list[0]
raw = rawpy.imread(fname)
ref_imarray = raw.raw_image_visible.astype(float)

In [ ]:
# masks that isolate the RGB pixels - these are camera-dependent and work with all images
colors_array = raw.raw_colors_visible

red_mask = np.where(colors_array == 0, 1, 0)

green_mask_1 = np.where(colors_array == 1, 1, 0)
green_mask_2 = np.where(colors_array == 3, 1, 0)
green_mask = green_mask_1 | green_mask_2

blue_mask = np.where(colors_array == 2, 1, 0)

In [ ]:
# normalization factors for the RGB arrays
# red_norm = 1.321875  # smooth background  
# green_norm = 1.
# blue_norm = 1.27695312

red_norm = 1.9  # spectrum
green_norm = 1.
blue_norm = 1.9

Using normalizations derived from passband spectral response is *much* better than using normalizations derived from minimization of sky background variance (contrary to the initial finding in notebook Offsets_1). The likely cause is that the spectral-based normalization creates more well-behaved star images. The high variance in sky background doesn't seem to get in the way of detecting stars.

The best run with smooth background generated 240 detections with a complete data set. The same run but with spectral-based color band normalizations resulted in 550 detections.

## Processing functions

In [ ]:
# computes position offsets between two tables. 
def get_offsets(sources, sources_prev):

    sources.add_column(np.nan, name='xoffset')
    sources.add_column(np.nan, name='yoffset')
    sources.add_column(0.0, name='xoffset_prev')
    sources.add_column(0.0, name='yoffset_prev')
    sources.add_column(0, name='ref_row')
    sources.add_column(0, name='prev_row')

    # loop over rows in previous table
    for row_index_prev in range(len(sources_prev)):
        # index in reference table
        ref_row = sources_prev[row_index_prev]['ref_row']

        # if previous table does not contain a pointer to 
        # the reference table, ignore.
        if ref_row == 0:
            continue

        # get position in previous table
        x_prev = sources_prev[row_index_prev]['xcentroid']
        y_prev = sources_prev[row_index_prev]['ycentroid']

        # loop over rows in current table
        for row_index in range(len(sources)):
            x = sources[row_index]['xcentroid']
            y = sources[row_index]['ycentroid']

            # offsets in relation to previous table - these are the ones to check for proximity
            x_off_previous = x - x_prev
            y_off_previous = y - y_prev

            # check for proximity, and store relevant info if found
            if abs(x_off_previous) <= proximity and abs(y_off_previous) <= proximity:

                # offsets in relation to reference table
                sources[row_index]['xoffset'] = x - sources_ref[ref_row]['xcentroid']
                sources[row_index]['yoffset'] = y - sources_ref[ref_row]['ycentroid']

                # offsets in relation to previous table
                sources[row_index]['xoffset_prev'] = x_off_previous
                sources[row_index]['yoffset_prev'] = y_off_previous

                # store pointers to rows in reference and previous tables
                sources[row_index]['ref_row'] = ref_row
                sources[row_index]['prev_row'] = row_index_prev
                
                #TODO 
                # instead of breaking, do an estimate of where the centroid would be,
                # given the current position, and the offsets from the previous table.
                # In other words, repeat the offset from the previous table. See if this
                # will cause the finding algorithm to pick up in the next image.

                break # if there is another star that matches the criterion, just ignore it
                
    return sources

In [ ]:
# creates a table with star positions, given a path to an image file
def find_stars(path, sources_prev=None):

    with rawpy.imread(path) as raw:
        imarray = raw.raw_image_visible.astype(float)

        # normalize
        raw_norm_1 = imarray * (red_mask * red_norm)
        raw_norm_2 = raw_norm_1 + imarray * (green_mask * 1.0)
        raw_norm = raw_norm_2 + imarray * (blue_mask * blue_norm)

        # handle saturated pixels
        raw_norm = np.where(imarray > 16380, imarray, raw_norm)

        # compute and subtract background
        bkg = Background2D(raw_norm, bkg_cell_footprint, filter_size=bkg_filter, sigma_clip=sigma_clip, bkg_estimator=bkg_estimator)
        subtracted = raw_norm - bkg.background

        # find stars
        if sources_prev is None:
            daofind = DAOStarFinder(fwhm=dao_fwhm, threshold=dao_threshold * bkg.background_rms_median) 
        else:
            # offsets are added in reverse, to generate an estimate further away from the current position.
            x_estimate = sources_prev['xcentroid'] - sources_prev['xoffset_prev'] 
            y_estimate = sources_prev['ycentroid'] - sources_prev['yoffset_prev'] 
            positions = [(x,y) for x,y in zip(x_estimate, y_estimate)]
            daofind = DAOStarFinder(xycoords=np.array(positions), fwhm=dao_fwhm, 
                                    threshold=dao_threshold * bkg.background_rms_median) 
        sources = daofind(subtracted)

        return sources, subtracted

In [ ]:
# keep only the NaN-free entries
def clean_nans(sources):
    has_nan = np.zeros(len(sources), dtype=bool)
    xoff = np.array(sources['xoffset'])
    has_nan |= np.isnan(xoff)
    return sources[~has_nan]

## Process sequence

In [ ]:
# find stars in reference image
files = sort(file_list)
sources_ref, subtracted_ref = find_stars(files[0])

# positions storage
positions_tables = {}
positions_tables[files[0]] = sources_ref

# array for stacking subtracted images
image_stack = np.zeros_like(subtracted_ref)

# add default offset columns to reference table
sources_ref.add_column(0., name='xoffset')
sources_ref.add_column(0., name='yoffset')
sources_ref.add_column(0., name='xoffset_prev')
sources_ref.add_column(0., name='yoffset_prev')

# in ref table, rows point to themselves
sources_ref.add_column(sources_ref['id']-1, name='ref_row')
sources_ref.add_column(sources_ref['id']-1, name='prev_row')

# force reference image to be the "previous" image
sources_prev = sources_ref

sources_ref

In [ ]:
# loop over list of remaining images
for file_path in files[1:]:
    
    # find stars
    sources, subtracted = find_stars(file_path, sources_prev=None)
    
    # compute offsets
    sources_current = get_offsets(sources, sources_prev)
    
    sources_current_no_nan = clean_nans(sources_current)
    
    positions_tables[file_path] = sources_current_no_nan
    
    # save table to file
    
    # for next iteration, current table becomes previous
    sources_prev = sources_current_no_nan
    
    # update image stack
    image_stack += subtracted
    
    print(file_path, len(sources_current_no_nan))

In [ ]:
sources_current_no_nan

In [ ]:
# positions = [(x,y) for x,y in zip(sources_current_no_nan['xcentroid'], sources_current_no_nan['ycentroid'])]
# positions_ref = [(x,y) for x,y in zip(sources_ref['xcentroid'], sources_ref['ycentroid'])]

# apertures = CircularAperture(positions, r=5.)
# apertures_ref = CircularAperture(positions_ref, r=5.)

# plt.figure(figsize=[9, 6])
# plt.imshow(image_stack, vmin=-10, vmax=10000, cmap='binary')
# plt.colorbar()
# _ = apertures.plot(color='red')
# _ = apertures_ref.plot(color='yellow')

In [ ]:
plt.figure(figsize=[9, 6])
plt.imshow(image_stack, vmin=-10, vmax=30000, cmap='binary')
plt.colorbar()

for file_path in list(positions_tables.keys()):
    positions_t = positions_tables[file_path]
    positions = [(x,y) for x,y in zip(positions_t['xcentroid'], positions_t['ycentroid'])]
    apertures = CircularAperture(positions, r=1.)
    _ = apertures.plot(color='red')

## Keep only complete stars in sequence

Stars with a complete sequence of measured centroids are the ones that make to the last image/table in the sequence. Thus, starting from the end image and going backwards, we ensure we pick only the complete sequence stars.

In [ ]:
# start from last table in sequence 
file_path_last = list(positions_tables.keys())[-1]
positions_last = positions_tables[file_path_last]
refrow_last = positions_last['ref_row']

file_path_first = list(positions_tables.keys())[0]  # ref table
positions_first = positions_tables[file_path_first]

plt.figure(figsize=[9, 6])
plt.imshow(image_stack, vmin=-10, vmax=30000, cmap='binary')
plt.colorbar()

for i, row in enumerate(refrow_last):
    ref_row = positions_first[row]
    
    x0 = ref_row['xcentroid']
    y0 = ref_row['ycentroid']
    x1 = positions_last['xcentroid'][i]
    y1 = positions_last['ycentroid'][i]
    
    plot([x0,x1], [y0,y1], 'r', linewidth=1, markersize=1)

## Write tables

In [ ]:
keys = list(positions_tables.keys())

for key in keys:
    dirname = os.path.dirname(key)
    fname = os.path.basename(key)
    
    # get image time and add to table header
    f = open( key, 'rb')
    tags = exifread.process_file(f)
    date_time = tags['EXIF DateTimeOriginal']
    
    imagename = fname.split('.')[0]
    tablename = os.path.join(dirname, imagename + '.offsets_table.fits')
    
    table = positions_tables[key]

    table.write(tablename, format='fits', overwrite=True)

    print(date_time, tablename)